In [2]:
#pip install kaggle
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

In [3]:
data_dir = "data"
!mkdir $data_dir 

#!cd $data_dir && kaggle datasets download netflix-inc/netflix-prize-data/ 
#!cd $data_dir && unzip netflix-prize-data.zip
#dataset_dir = data_dir + "/netflix-prize-data/"
#!ls $dataset_dir

Un sous-r�pertoire ou un fichier data existe d�j�.


In [6]:
# Définition d'une fonction de conversion de format des données sources
# .txt à 2 niveaux => .csv plat (1 niveau)
def convert_data(source, destination):
    id =""
    #open file1 in reading mode
    dirty_file = open(source, 'r')
    
    #open file2 in writing mode
    cleaned_file = open(destination,'w')
    
    #read from file1 and write to file2
    for line in dirty_file:
        line = line.strip()
        if ":" in  line:
            id = line[:-1]
        else:
            element = line.split(',')
            timestamp = int(time.mktime(datetime.strptime(element[-1],"%Y-%m-%d").timetuple()))
            line += ","+ id +  "," + str(timestamp) + "\n" 
            cleaned_file.write(line)
    
    #close file1 and file2
    dirty_file.close()
    cleaned_file.close()



In [7]:
# Conversion des 4 fichiers sources
#convert_data(data_dir + '/combined_data_1.txt',data_dir + '/data_1.csv')
#convert_data(data_dir + '/combined_data_2.txt',data_dir + '/data_2.csv')
#convert_data(data_dir + '/combined_data_3.txt',data_dir + '/data_3.csv')
#convert_data(data_dir + '/combined_data_4.txt',data_dir + '/data_4.csv')

In [4]:
# Connexion AWS
import boto3
aws_session = boto3.Session(
    aws_access_key_id="",
    aws_secret_access_key="",
    region_name=""
    )

num_data=4

In [112]:
# Selection des attributs du dataset
data_files = ['/data_1.csv','/data_2.csv','/data_3.csv','/data_4.csv'] # A faire successivement pour chaque fichier de données (data_1.csv à data_4.csv) 
original_data = pd.read_csv(data_dir + data_files[num_data-1], header = None, usecols = [0,3,1,4], names = ['userId','rating','movieId','timestamp'])
print(original_data.info())
original_data.head()

#data_1 : 24053764 entries
#data_2 : 26977591 entries
#data_3 : 22601629 entries
#data_4 : 26847523 entries



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26847523 entries, 0 to 26847522
Data columns (total 4 columns):
 #   Column     Dtype
---  ------     -----
 0   userId     int64
 1   rating     int64
 2   movieId    int64
 3   timestamp  int64
dtypes: int64(4)
memory usage: 819.3 MB
None


,userId,rating,movieId,timestamp
0,2385003,4,13368,1089237600
1,659432,3,13368,1110927600
2,751812,2,13368,1039993200
3,2625420,2,13368,1085436000
4,1650301,1,13368,1125352800


définition deux variables dans l'ensemble de données pour filtrer les films non appréciés et mieux simuler les données recueillies par une plateforme de vidéo à la demande (VOD).

Puisqu'il s'agit d'un ensemble de données d'évaluation explicite de films, il comprend des films évalués de 1 à 5. Nous voulons inclure uniquement les mouvements qui ont été "aimés" par les utilisateurs, et simuler un ensemble de données implicites qui est similaire aux données qui sont recueillies par une plate-forme de netflix. Pour cela, nous allons ensuite filtrer toutes les interactions inférieures à 2 sur 5, et créer deux variables EVENT_TYPE : click et watch. Tous les films notés 2 et plus sont affectés à la catégorie "cliquer", et tous les films notés 4 et plus sont affectés à la catégorie "cliquer et regarder".

In [113]:
# Préparation du dataset

watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] >= 4]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'

clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] >= 2]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'

interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, inplace = True, na_position ='last')

print(interactions_df.info())
interactions_df.head()

#interactions_df for data_1 : 36527902 entries
#interactions_df for data_2 : 40842631 entries
#interactions_df for data_3 : 34354528 entries
#interactions_df for data_4 : 41056646 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41056646 entries, 8260224 to 12379098
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   userId      int64 
 1   movieId     int64 
 2   timestamp   int64 
 3   EVENT_TYPE  object
dtypes: int64(3), object(1)
memory usage: 1.5+ GB
None


,userId,movieId,timestamp,EVENT_TYPE
8260224,510180,14660,942274800,click
19392456,510180,16465,942274800,click
26827865,510180,17764,942274800,click
11119861,510180,15105,942274800,click
10669666,510180,15057,942274800,click


save the interaction_df set. We upload this set to an Amazon S3 bucket in a later step

In [114]:
#Durée : 1 mn
interactions_filenames =['interactions_1.csv','interactions_2.csv','interactions_3.csv','interactions_4.csv']
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 'timestamp':'TIMESTAMP'}, inplace = True) 
interactions_filename = interactions_filenames[num_data-1] # A faire pour générer chaque fichier d'interactions (interactions_1.csv à interactions_4.csv)
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

In [5]:
#Création du dataset group
data_set_group_name = "personalize-demo-movielens"+str(num_data) # A faire pour créer chaque modèle (personalize-demo-movielens1 à personalize-demo-movielens4)

# Create a client session for presonnalize
personalize = aws_session.client('personalize')

#  Configure the SDK to Personalize:
personalize_runtime = aws_session.client('personalize-runtime')

create_dataset_group_response = personalize.create_dataset_group(name = data_set_group_name)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
# dataset_group_arn = "arn:aws:personalize:eu-west-1:461868250861:dataset-group/personalize-demo-movielens1" # Data set 1
# dataset_group_arn = "arn:aws:personalize:eu-west-1:461868250861:dataset-group/personalize-demo-movielens2" # Data set 2
# dataset_group_arn = "arn:aws:personalize:eu-west-1:461868250861:dataset-group/personalize-demo-movielens3" # Data set 3
# dataset_group_arn = "arn:aws:personalize:eu-west-1:461868250861:dataset-group/personalize-demo-movielens4" # Data set 4

#print(dataset_group_arn)
print(json.dumps(create_dataset_group_response, indent=2))


ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateDatasetGroup operation: Another resource with Arn arn:aws:personalize:eu-west-1:461868250861:dataset-group/personalize-demo-movielens4 already exists.

In [116]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

DatasetGroup: ACTIVE
Wall time: 114 ms


Amazon Personalize needs a schema to understand your data. The following code block creates the appropriate schema for the Movie dataset and provides it to Personalize. This code block also creates the interactions dataset within the dataset group. Personalize uses this dataset to train the recommendation model.

In [117]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = data_set_group_name + "-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
#interaction_schema_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens1-interactions" # Data set 1
#interaction_schema_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens2-interactions" # Data set 2
#interaction_schema_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens3-interactions" # Data set 3
#interaction_schema_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens4-interactions" # Data set 4
print(json.dumps(create_schema_response, indent=2))

dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = data_set_group_name + "-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)
interactions_dataset_arn = create_dataset_response['datasetArn']
#interactions_dataset_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens1-interactions" # Data set 1
#interactions_dataset_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens2-interactions" # Data set 2
#interactions_dataset_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens3-interactions" # Data set 3
#interactions_dataset_arn = "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens4-interactions" # Data set 4
print(json.dumps(create_dataset_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:eu-west-1:461868250861:schema/personalize-demo-movielens4-interactions",
  "ResponseMetadata": {
    "RequestId": "d747535e-c5c2-43b7-b2f4-c927d7f5deb3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 18 Jul 2022 22:37:34 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "106",
      "connection": "keep-alive",
      "x-amzn-requestid": "d747535e-c5c2-43b7-b2f4-c927d7f5deb3"
    },
    "RetryAttempts": 0
  }
}
{
  "datasetArn": "arn:aws:personalize:eu-west-1:461868250861:dataset/personalize-demo-movielens4/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "cbf6160c-fc50-4b66-8b49-15e21ce45300",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 18 Jul 2022 22:37:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "cbf6160c-fc50-4b66-8b49-15e21ce45300"
    },
    "RetryAttempt

Training a model produces model training data and model artifacts. In this lab, you use an Amazon S3 bucket to stage the interactions dataset, and store the model artifacts generated by Amazon Personalize during model training.

In your Jupyter notebook, copy and paste the following code into a new code cell and choose Run.

In [118]:
region = aws_session.region_name
s3 = aws_session.client('s3')
account_id = aws_session.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizedemoml"
print(bucket_name)
if region == "eu-west-3":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )

461868250861-eu-west-1-personalizedemoml


BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [22]:
#Ne pas ituliser !!
# Déposer les fichiers d'interactions directement sur S3

#interactions_file_path = data_dir + "/" + interactions_filename
#aws_session.resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
#interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

now we configure the Amazon S3 bucket policy so that Amazon Persoytrnalize can read the content of your S3 bucket.

In [119]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'GVFG4Z11DM3TEM2W',
  'HostId': '3iDNzb1eLkhF0nMP7/F/ZeN4fBFZVgu3ewbxeskaBQZTksVw5+1gX440Wt/zZvt1M/bt2yqDEXc=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '3iDNzb1eLkhF0nMP7/F/ZeN4fBFZVgu3ewbxeskaBQZTksVw5+1gX440Wt/zZvt1M/bt2yqDEXc=',
   'x-amz-request-id': 'GVFG4Z11DM3TEM2W',
   'date': 'Mon, 18 Jul 2022 22:38:01 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [61]:
iam = aws_session.client("iam")

role_name = "PersonalizeRolePOC"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
#role_arn = "arn:aws:iam::461868250861:role/PersonalizeRolePOC"
print(role_arn)

arn:aws:iam::461868250861:role/PersonalizeRolePOC


Remember that you created the dataset group and dataset in Step 2. Now, you can create the import job that loads the data from Amazon S3 into Amazon Personalize to use for your model.

In [121]:
#Création du job d'import du dataset
role_arn = "arn:aws:iam::461868250861:role/PersonalizeRolePOC"

create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import"+str(num_data),
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
#datasetImportJobArn="arn:aws:personalize:eu-west-1:461868250861:dataset-import-job/personalize-demo-import1"
#datasetImportJobArn="arn:aws:personalize:eu-west-1:461868250861:dataset-import-job/personalize-demo-import2"
#datasetImportJobArn="arn:aws:personalize:eu-west-1:461868250861:dataset-import-job/personalize-demo-import3"
#datasetImportJobArn="arn:aws:personalize:eu-west-1:461868250861:dataset-import-job/personalize-demo-import4"
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:eu-west-1:461868250861:dataset-import-job/personalize-demo-import4",
  "ResponseMetadata": {
    "RequestId": "30bef2b9-6495-4cc5-a5c2-4b883e91a4e9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 18 Jul 2022 23:18:33 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "112",
      "connection": "keep-alive",
      "x-amzn-requestid": "30bef2b9-6495-4cc5-a5c2-4b883e91a4e9"
    },
    "RetryAttempts": 0
  }
}


In [122]:
%%time
# Durée : 3 à 4 mn
max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(10)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
D

In [123]:
# aws-user-personalization selected for project
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

In [124]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-user-personalization"+str(num_data),
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
#solutionArn = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization1"
#solutionArn = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization2"
#solutionArn = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization3"
#solutionArn = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization4"
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization4",
  "ResponseMetadata": {
    "RequestId": "76d943e3-e6a2-4208-9fb0-5605bcb07c19",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 18 Jul 2022 23:23:43 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "76d943e3-e6a2-4208-9fb0-5605bcb07c19"
    },
    "RetryAttempts": 0
  }
}


In [125]:
solution_version_arn = ["","","",""]
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn[num_data-1] = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization4/b600e60e",
  "ResponseMetadata": {
    "RequestId": "320e8435-012f-46ac-bd44-f80e27c26359",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 18 Jul 2022 23:24:02 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "129",
      "connection": "keep-alive",
      "x-amzn-requestid": "320e8435-012f-46ac-bd44-f80e27c26359"
    },
    "RetryAttempts": 0
  }
}


In [70]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn[num_data-1]
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


KeyboardInterrupt: 

In [11]:
# Affichage des métriques pour évaluer les versions de solutions
solution_version_arn = ["","","",""]
solution_version_arn[0] = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization1/ce32ca4d"
solution_version_arn[1] = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization2/da4eaf5a"
solution_version_arn[2] = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization3/7ba6d4c3"
solution_version_arn[3] = "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization4/b600e60e"
sol_metrics_df = None

for i in range(4):
    get_solution_metrics_response = personalize.get_solution_metrics(
        solutionVersionArn = solution_version_arn[i]
    )
    #TODO : convertir les metrics en df pour afficher les 4 campagnes et exporter dans G-Sheets
    metrics = get_solution_metrics_response["metrics"]
    print(metrics)
    
    print(json.dumps(get_solution_metrics_response, indent=2))


{'coverage': 0.6393, 'mean_reciprocal_rank_at_25': 0.4497, 'normalized_discounted_cumulative_gain_at_10': 0.449, 'normalized_discounted_cumulative_gain_at_25': 0.5037, 'normalized_discounted_cumulative_gain_at_5': 0.4088, 'precision_at_10': 0.0911, 'precision_at_25': 0.0533, 'precision_at_5': 0.137}
{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:461868250861:solution/personalize-demo-soln-user-personalization1/ce32ca4d",
  "metrics": {
    "coverage": 0.6393,
    "mean_reciprocal_rank_at_25": 0.4497,
    "normalized_discounted_cumulative_gain_at_10": 0.449,
    "normalized_discounted_cumulative_gain_at_25": 0.5037,
    "normalized_discounted_cumulative_gain_at_5": 0.4088,
    "precision_at_10": 0.0911,
    "precision_at_25": 0.0533,
    "precision_at_5": 0.137
  },
  "ResponseMetadata": {
    "RequestId": "7f134571-d7b7-4423-bbc6-3cf3dc3410f0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 19 Jul 2022 11:13:02 GMT",
      "content-type": "application/x-a

In [143]:
#Création d'une campagne pour déployer une version de solution
#Durée : 10 mn
num_data=4
campaign_arn = ["","","",""]
create_campaign_response = personalize.create_campaign(
    name = "personalize-demo-camp"+str(num_data),
    solutionVersionArn = solution_version_arn[num_data-1],
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.3",
	        "explorationItemAgeCutOff": "30"
        }
    }
)

campaign_arn[num_data-1] = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn[num_data-1]
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

{
  "campaignArn": "arn:aws:personalize:eu-west-1:461868250861:campaign/personalize-demo-camp4",
  "ResponseMetadata": {
    "RequestId": "6b71b873-29d1-4280-898b-9e2ef26ec3ea",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 19 Jul 2022 06:50:42 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "92",
      "connection": "keep-alive",
      "x-amzn-requestid": "6b71b873-29d1-4280-898b-9e2ef26ec3ea"
    },
    "RetryAttempts": 0
  }
}
Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


In [7]:
# Test de la campagne
campaign_arn = ["","","",""]
campaign_arn[0] = "arn:aws:personalize:eu-west-1:461868250861:campaign/personalize-demo-camp1"
campaign_arn[1] = "arn:aws:personalize:eu-west-1:461868250861:campaign/personalize-demo-camp2"
campaign_arn[2] = "arn:aws:personalize:eu-west-1:461868250861:campaign/personalize-demo-camp3"
campaign_arn[3] = "arn:aws:personalize:eu-west-1:461868250861:campaign/personalize-demo-camp4"

# Build a map to convert a movie id to the movie title
movies = pd.read_csv('./data/movie_titles.csv', usecols=[0,2], names=["movieId","mivieTittle"], encoding='utf-8')
movies['movieId'] = movies['movieId'].astype(str)
movie_map = dict(movies.values)

# Getting a random user:
user_id=605662
#user_id=1043824

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn[num_data-1],
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']
print(item_list)
recommendation_list = []
for item in item_list:
    movie = movie_map[item['itemId']]
    score = item['score']
    recommendation_list.append([movie, score])

recommendations_df = pd.DataFrame(recommendation_list, columns = ['Movie','Score'])
recommendations_df.head(30)

Recommendations for user:  605662
[{'itemId': '15124', 'score': 0.0197101}, {'itemId': '14313', 'score': 0.01816}, {'itemId': '15582', 'score': 0.014212}, {'itemId': '16242', 'score': 0.014017}, {'itemId': '14538', 'score': 0.0128889}, {'itemId': '14454', 'score': 0.0122038}, {'itemId': '15107', 'score': 0.010477}, {'itemId': '16377', 'score': 0.0103793}, {'itemId': '15205', 'score': 0.01032}, {'itemId': '14367', 'score': 0.0095456}, {'itemId': '16879', 'score': 0.00867}, {'itemId': '15788', 'score': 0.0086563}, {'itemId': '13728', 'score': 0.0084645}, {'itemId': '14712', 'score': 0.0084246}, {'itemId': '14410', 'score': 0.0082779}, {'itemId': '13651', 'score': 0.0081263}, {'itemId': '14691', 'score': 0.0080985}, {'itemId': '14240', 'score': 0.0076044}, {'itemId': '15563', 'score': 0.0073282}, {'itemId': '13462', 'score': 0.0069292}, {'itemId': '17355', 'score': 0.0067112}, {'itemId': '16082', 'score': 0.0066262}, {'itemId': '14149', 'score': 0.0064494}, {'itemId': '15471', 'score': 0.

,Movie,Score
0,Independence Day,0.019710
1,The Patriot,0.018160
2,Sweet Home Alabama,0.014212
3,Con Air,0.014017
4,How to Lose a Guy in 10 Days,0.012889
5,Kill Bill: Vol. 1,0.012204
6,Ocean's Eleven,0.010477
7,The Green Mile,0.010379
8,The Day After Tomorrow,0.010320
9,Lethal Weapon 4,0.009546
